```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("db-connection-2")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "10gb")
    .config("spark.jars", "postgresql-42.2.24.jar")
    .getOrCreate()
)

21/11/22 14:18:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
properties = {"user": "", "password": "", "host": "", "port": "", "database": ""}
properties["url"] = (
    "jdbc:postgresql://"
    + properties["host"]
    + ":"
    + properties["port"]
    + "/"
    + properties["database"]
)

In [ ]:
jdbcDF = (
    spark.read.format("jdbc")
    .options(
        url=properties["url"],  # jdbc:postgresql://<host>:<port>/<database>
        dbtable="clima.t_indices_prec_cpc",
        user=properties["user"],
        password=properties["password"],
        driver="org.postgresql.Driver",
    )
    .load()
)

In [ ]:
jdbcDF.printSchema()

root
 |-- id_pk: integer (nullable = true)
 |-- codigo: integer (nullable = true)
 |-- ano: integer (nullable = true)
 |-- cdd: decimal(10,2) (nullable = true)
 |-- prcptot: decimal(10,2) (nullable = true)
 |-- sdii: decimal(10,2) (nullable = true)
 |-- r20mm: decimal(10,2) (nullable = true)
 |-- r30mm: decimal(10,2) (nullable = true)
 |-- r50mm: decimal(10,2) (nullable = true)
 |-- r80mm: decimal(10,2) (nullable = true)
 |-- r100mm: decimal(10,2) (nullable = true)
 |-- r150mm: decimal(10,2) (nullable = true)
 |-- rx1day: decimal(10,2) (nullable = true)
 |-- rx2day: decimal(10,2) (nullable = true)
 |-- rx5day: decimal(10,2) (nullable = true)



In [ ]:
%%timeit
jdbcDF.filter("r100mm > 2000")

1.15 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
spark.stop()

In [ ]:
! pip install psycopg2-binary

In [ ]:
import psycopg2


connection = psycopg2.connect(
    user=properties["user"],
    password=properties["password"],
    host=properties["host"],
    port=properties["port"],
    database=properties["database"],
)

In [ ]:
%%timeit
from pandas import DataFrame

cursor = connection.cursor()
cursor.execute("SELECT * FROM clima.t_indices_prec_cpc t Where r100mm > 2000")
names = [x[0] for x in cursor.description]
result = cursor.fetchall()
df = DataFrame(result, columns=names)

2.57 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
connection.close()